In [323]:
import polars as pl

In [324]:
df = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv')
df

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zuverge01""",1958,1,"""BAL""","""AL""",2,2,45,0,0,0,7,207,74,26,4,17,22,0.28,3.39,3,2,6,0,294,23,29,null,null,null
"""zuverge01""",1959,1,"""BAL""","""AL""",0,1,6,0,0,0,0,39,15,6,1,6,1,0.3,4.15,0,1,0,0,55,1,7,null,null,null
"""zychto01""",2015,1,"""SEA""","""AL""",0,0,13,1,0,0,0,55,17,5,1,3,24,0.239,2.45,0,1,2,0,76,4,6,0,0,0


In [325]:
df = (
    df
    .select(['yearID', 'lgID', 'R', 'IPouts'])
    .filter(pl.col('yearID') > 1945)
    .filter((pl.col('lgID') == 'AL') | (pl.col('lgID') == 'NL'))
    .group_by(['lgID', 'yearID'])
    .agg(pl.col("R").sum().alias("Total_Runs"), pl.col("IPouts").sum().alias("Total_Outs"))
)

df

lgID,yearID,Total_Runs,Total_Outs
str,i64,i64,i64
"""AL""",1956,5756,33095
"""AL""",2020,4159,23338
"""AL""",1979,10527,60155
"""AL""",2011,10036,60850
"""AL""",1992,9802,60987
…,…,…,…
"""AL""",2017,11337,64892
"""AL""",1990,9746,60547
"""AL""",1949,5776,32823


In [326]:
df = (
    df
    .with_columns([((pl.col("Total_Runs") / pl.col("Total_Outs")) * 27).alias("ERA")])
   .sort(pl.col('yearID'))
    )

df

lgID,yearID,Total_Runs,Total_Outs,ERA
str,i64,i64,i64,f64
"""AL""",1946,5037,33186,4.098084
"""NL""",1946,4918,33254,3.993084
"""AL""",1947,5161,33228,4.193662
"""NL""",1947,5669,32902,4.652088
"""AL""",1948,5838,32929,4.786844
…,…,…,…,…
"""NL""",2021,10869,63797,4.59995
"""NL""",2022,10602,64550,4.434609
"""AL""",2022,10215,64676,4.26441


In [329]:
df = (
    df.with_columns(
        pl
        .when((pl.col('lgID') == 'NL') & (pl.col('yearID').is_in(2020) | pl.col('yearID') >= 2022))
            .then(pl.lit('Yes'))
        .when((pl.col('lgID') == 'AL') & (pl.col('yearID') >= 1973))
            .then(pl.lit('Yes'))
        .otherwise(pl.lit('No'))
        .alias('DH')
    )
)

print(df.head())

shape: (5, 6)
┌──────┬────────┬────────────┬────────────┬──────────┬─────┐
│ lgID ┆ yearID ┆ Total_Runs ┆ Total_Outs ┆ ERA      ┆ DH  │
│ ---  ┆ ---    ┆ ---        ┆ ---        ┆ ---      ┆ --- │
│ str  ┆ i64    ┆ i64        ┆ i64        ┆ f64      ┆ str │
╞══════╪════════╪════════════╪════════════╪══════════╪═════╡
│ AL   ┆ 1946   ┆ 5037       ┆ 33186      ┆ 4.098084 ┆ No  │
│ NL   ┆ 1946   ┆ 4918       ┆ 33254      ┆ 3.993084 ┆ No  │
│ AL   ┆ 1947   ┆ 5161       ┆ 33228      ┆ 4.193662 ┆ No  │
│ NL   ┆ 1947   ┆ 5669       ┆ 32902      ┆ 4.652088 ┆ No  │
│ AL   ┆ 1948   ┆ 5838       ┆ 32929      ┆ 4.786844 ┆ No  │
└──────┴────────┴────────────┴────────────┴──────────┴─────┘


In [328]:
df.write_csv('./Data/ERA_Data.csv', separator=",")